In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 215.7MB 48kB/s 
     |████████████████████████████████| 204kB 42.2MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=a8895c080aec6e76cb15ce3e2c7ed1571c97550c6fd358108a175e10b3010e0d
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark
openjdk-8-jdk-headless is already the newest version (8u232-b09-0ubuntu1~18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [0]:
data = sc.textFile('soc-LiveJournal1Adj.txt')

In [0]:
def make_tuple(line):
  tokens = line.split('\t')

  if len(tokens) == 1 or tokens[0] == '':
    return []

  user = int(tokens[0])
  friends = tokens[1].split(',')

  user_friend_pairs = []
  mutual_friend_pairs = []

  for friend in friends:
    if friend != '':
      user_friend_pairs.append(((user, int(friend)), 0))

  for i in range(0, len(friends) - 1):
    for j in range(i + 1, len(friends)):
      mutual_friend_pairs.append(((int(friends[i]), int(friends[j])), 1))
      mutual_friend_pairs.append(((int(friends[j]), int(friends[i])), 1))

  return user_friend_pairs + mutual_friend_pairs

In [0]:
relationship_pairs = data.flatMap(lambda line: make_tuple(line))

In [0]:
already_friends = relationship_pairs.filter(lambda relationship: relationship[1] == 0)

In [0]:
mutual_friends_pairs = relationship_pairs.subtractByKey(already_friends).\
                       reduceByKey(lambda a, b: a + b).\
                       map(lambda x: (x[0][0], (x[0][1], x[1]))).\
                       groupByKey().\
                       mapValues(list)

In [0]:
def recommend_new_friends(user_and_strangers, n = 10):
  user, list_of_strangers = user_and_strangers

  ordered_strangers_list = sorted(list_of_strangers, key = lambda stranger_num_mutual_friends: (-stranger_num_mutual_friends[1], stranger_num_mutual_friends[0]))[:n]

  #recommendation = map(lambda stranger_num_mutual_friends: stranger_num_mutual_friends[0], ordered_strangers_list)
  recommendation = []
  for k, v in ordered_strangers_list:
    recommendation.append(k)

  return user, recommendation

In [0]:
result = mutual_friends_pairs.map(lambda user_and_strangers: recommend_new_friends(user_and_strangers)).\
         map(lambda user_recommendations: "{}\t{}".format(user_recommendations[0], ",".join(map(lambda x: str(x), user_recommendations[1])))).\
         collect()

In [135]:
user_IDs = ['924', '8941', '8942', '9019', '9020', '9021', '9022', '9990', '9992', '9993']

for user_id in user_IDs:
  for line in result:
    user, recommendations = line.split('\t')
    if user == user_id:
      print(line)

924	439,2409,6995,11860,15416,43748,45881
8941	8943,8944,8940
8942	8939,8940,8943,8944
9019	9022,317,9023
9020	9021,9016,9017,9022,317,9023
9021	9020,9016,9017,9022,317,9023
9022	9019,9020,9021,317,9016,9017,9023
9990	13134,13478,13877,34299,34485,34642,37941
9992	9987,9989,35667,9991
9993	9991,13134,13478,13877,34299,34485,34642,37941
